In [70]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [30]:
df=pd.read_csv('twitter_training.csv', names=['ID','Platforms','Sentiments','Tweet'])
df.head()

,ID,Platforms,Sentiments,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [31]:
df.drop_duplicates(inplace=True)

In [32]:
df.duplicated().sum()

0

In [33]:
df.isna().sum()

ID              0
Platforms       0
Sentiments      0
Tweet         326
dtype: int64

In [34]:
df.shape

(71982, 4)

In [35]:
df.dropna(axis=0, inplace=True)

In [36]:
df.shape

(71656, 4)

In [37]:
df['Sentiments'].value_counts()

Sentiments
Negative      21698
Positive      19713
Neutral       17708
Irrelevant    12537
Name: count, dtype: int64

In [82]:

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_and_stem(text):
        # Remove URLs, mentions, hashtags, and special characters
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    tokens = text.split(' ')
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

In [83]:
df['Tweet']=df['Tweet'].apply(clean_and_stem)

In [84]:
df.head()

,ID,Platforms,Sentiments,Tweet
0,2401,Borderlands,Positive,im get borderland murder
1,2401,Borderlands,Positive,come border kill
2,2401,Borderlands,Positive,im get borderland kill
3,2401,Borderlands,Positive,im come borderland murder
4,2401,Borderlands,Positive,im get borderland murder


In [85]:
x=df[['Tweet']]
y=df['Sentiments']

In [86]:
rus = RandomUnderSampler()

In [87]:
x_res,y_res=rus.fit_resample(x,y)

In [88]:
y_res.value_counts()

Sentiments
Irrelevant    12537
Negative      12537
Neutral       12537
Positive      12537
Name: count, dtype: int64

In [89]:
x_train,x_test,y_train,y_test = train_test_split(x_res,y_res, test_size=0.2)

In [90]:
vector = TfidfVectorizer()

In [91]:
x_train_vector = vector.fit_transform(x_train['Tweet'])
x_test_vector = vector.transform(x_test['Tweet'])

In [92]:
classifier = {
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression()
}

In [93]:
for name,model in classifier.items():
    print(f'============= { name } ===============\n')
    model.fit(x_train_vector,y_train)
    y_pred = model.predict(x_test_vector)
    print(f'Accuracy Score: {accuracy_score(y_pred,y_test)}')
    # print(f'Accuracy Score: {f1_score(y_pred,y_test)}')
    # print(f'Accuracy Score: {precision_score(y_pred,y_test)}')
    # print(f'Accuracy Score: {recall_score(y_pred,y_test)}')

============= Decision Tree ===============

Accuracy Score: 0.7615154536390828
============= Logistic Regression ===============

Accuracy Score: 0.7442671984047856


C:\Users\shani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [94]:
lr = LogisticRegression()
lr.fit(x_train_vector,y_train)

C:\Users\shani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [109]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train_vector,y_train)

DecisionTreeClassifier()

In [126]:
data=vector.transform(np.array(['I Love you']))
print(lr.predict(data)[0])
print(dtc.predict(data)[0])

Positive
Positive


In [117]:
import pickle

In [120]:
pickle.dump(lr,open('lr_model.pkl','wb'))

In [121]:
pickle.dump(dtc,open('dtc_model.pkl','wb'))

In [122]:
pickle.dump(vector,open('vector.pkl','wb'))